In [1]:
import numpy as np

# Exercise 1

Write a function $\texttt{mydet(A)}$ that calculates the determinate of a given $n \times n$ matrix $A$ using Laplace Expansion. 

In [2]:
A1 = np.array([[1,3,2,0],[1,1,2,3],[4,0,8,3],[2,5,4,5]])
A1, np.linalg.det(A1)

(array([[1, 3, 2, 0],
        [1, 1, 2, 3],
        [4, 0, 8, 3],
        [2, 5, 4, 5]]), 0.0)

In [3]:
A2 = np.array([[4,5,6,6],[7,7,9,-3],[8,4,0,-7],[-5,3,-6,-4]])
A2, np.linalg.det(A2)

(array([[ 4,  5,  6,  6],
        [ 7,  7,  9, -3],
        [ 8,  4,  0, -7],
        [-5,  3, -6, -4]]), -4485.0)

In [4]:
def mydet(A):
    # A should be a quadratic matrix. The following line checks that and if it is not true, aborts the function:
    # A.shape is a pair (nr. of rows, nr. of columns) -- we compare the two entries
    assert A.shape[0] == A.shape[1], "A not quadratic" 
    
    n = A.shape[0]
    if(n == 1):
        return A[0,0]
    
    det = 0
    for i in range(n):
        Ared = np.hstack((A[1:,:i], A[1:,i+1:]))
        det += (-1)**i * A[0,i] * mydet(Ared)
    return det














C= [1]

# Exercise 2

Write a function $\texttt{gauss(A,b)}$ that performs a Gaussian elimination on a $n \times m$ matrix $A$ and a $n \times 1$ vector $b$.

In [19]:
def gauss(A,b):
    
    n, m = A.shape
    assert n <= m
    
    for i in range(n):
        p = np.argmax(np.abs(A[i:,i])) + i
        b[[i,p]] = b[[p,i]]
        A[[i,p]] = A[[p,i]]
        
        if np.linalg.norm(A[i,:]) < 1e-15:
            b = np.hstack((b[:i], b[i+1:], b[i]))
            A = np.vstack((A[:i,:], A[i+1:,:], A[i,:]))
        
        for j in range(i+1,n):
            idx = i
            for jj in range(i,n):
                if abs(A[i,jj]) > 1e-15:
                    idx = jj
                    break
            if abs(A[i,idx]) > 1e-15:
                b[j] -= A[j,idx]/A[i,idx] * b[i]
                A[j,:] -= A[j,idx]/A[i,idx] * A[i,:]
    
    return A, b

In [20]:
A = np.array([[1.,2.,3.],[-3.,4.,-5.],[5.,6.,7.]])
A

array([[ 1.,  2.,  3.],
       [-3.,  4., -5.],
       [ 5.,  6.,  7.]])

In [21]:
b = np.array([1.,2.,3.])
b

array([1., 2., 3.])

In [22]:
gauss(A,b)

(array([[ 5.        ,  6.        ,  7.        ],
        [ 0.        ,  7.6       , -0.8       ],
        [ 0.        ,  0.        ,  1.68421053]]), array([3. , 3.8, 0. ]))

# Exercise 3

Write a function $\texttt{im_base(A)}$ that returns the basis of the image of a function $f$ for which $A$ is the corresponding matrix.

In [23]:
def im_base(A):
    #Because gauss(A) works in-place, we make a copy
    B = A.copy() 
    n, m = A.shape
    #Assertion is not needed, it is here for simplicity because we use gauss
    assert n <= m 
    A_red, _ = gauss(A, np.zeros(n))
    idx = []
    c = 0
    #Find the pivot-rows
    for i in range(m):
        if abs(A_red[i-c,i]) > 1e-16:
            idx.append(i)
        else:
            c += 1
    #Build the basis from the pivot-rows
    return B[:,idx]

In [25]:
A = np.array([[1.,2.,3.,4.],[2.,4.,7.,9.],[1.,2.,4.,5.]])
A

array([[1., 2., 3., 4.],
       [2., 4., 7., 9.],
       [1., 2., 4., 5.]])

In [26]:
im_base(A)

array([[1., 3.],
       [2., 7.],
       [1., 4.]])

# Exercise 4

Write a function rref(A) that returns the reduced row echelon form of a given matrix A.

In [12]:
def rref(B):
    A = B.copy()
    n, m = A.shape
    r = 0
    
    for c in range(m):
        ## Find the pivot row:
        pivot = np.argmax(np.abs(A[r:n,c])) + r
        if np.abs(A[pivot, c]) <= 1e-16:    
            ## Skip column c
            pass
        else:
            if pivot != r:
                ## Swap current row and pivot row
                A[[pivot, r], c:m] = A[[r, pivot], c:m]  
                
            ## Normalize pivot row
            A[r, c:m] = A[r, c:m] / A[r, c];

            ## Eliminate the current column
            v = A[r, c:m]
            
            ## Above (before row r):
            if r > 0:           
                ridx_above = np.arange(r)
                A[ridx_above, c:m] = A[ridx_above, c:m] - np.outer(v, A[ridx_above, c]).T
                
            ## Below (after row r):
            if r < n-1:
                ridx_below = np.arange(r+1,n)
                A[ridx_below, c:m] = A[ridx_below, c:m] - np.outer(v, A[ridx_below, c]).T
            r += 1
            
    ## Check if done
    if r == n:
        return A
    return A

In [13]:
A = np.array([[1.,2.,3.,4.],[2.,4.,7.,9.],[1.,2.,4.,5.]])
A[1] = A[1]
A

array([[1., 2., 3., 4.],
       [2., 4., 7., 9.],
       [1., 2., 4., 5.]])

In [14]:
rref(A)

array([[1., 2., 0., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 0.]])

# Exercise 5

Write a function ker_base(A) that returns the basis of the kernel of a function f for which A is the corresponding matrix.

In [11]:
def ker_base(A):
    n, m = A.shape
    #Bring A into reduced row echelon form
    A = rref(A)
    idx = []
    #Insert rows of [0,...,0,-1,0,...,0] at non-pivot rows
    for i in range(m):
        if abs(A[i,i]) < 1e-16:
            b = np.zeros(m)
            b[i] = -1
            A = np.insert(A,i,b,axis=0)
            idx.append(i)
    #Delete zero-rows
    N = A.shape[0]
    if N > m:
        del_idx = np.arange(m,N)
        A = np.delete(A, del_idx, axis=0)
    #Build the basis from the correct columns of A
    B = A[:,idx]
    return B

In [13]:
A = np.array([[1.,2.,3.,4.],[2.,4.,7.,9.],[1.,2.,4.,5.]])
A

array([[1., 2., 3., 4.],
       [2., 4., 7., 9.],
       [1., 2., 4., 5.]])

In [14]:
ker_base(A)

array([[ 2.,  1.],
       [-1.,  0.],
       [ 0.,  1.],
       [ 0., -1.]])